In [1]:
# Dependencies
import requests
import os
from os.path  import basename
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import datetime
from datetime import datetime
import time
from random import randint
import unicodedata


In [2]:
leagues_df = pd.read_csv("../data/leagues_data.csv")

In [3]:
leagues_df

,league_ID,country,tier,league_name,league_link
0,1,Brazil,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...
1,2,United States,1,MLS (US),https://www.transfermarkt.com/major-league-soc...
2,3,Mexico,1,Liga MX (MEX),https://www.transfermarkt.com/liga-mx-clausura...
3,4,Argentina,1,Superliga (ARG),https://www.transfermarkt.com/primera-division...


In [4]:
# to be able to scrape this website we need to use 'User Agents'
# more info about user agents in 'https://webscraping.com/blog/User-agents/'
# You can find your User-Agent at 'http://whatsmyuseragent.com/'
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

In [5]:
convert_url = "https://www.google.com/search?q=euro+to+dollar&oq=eur&aqs=chrome.1.69i57j35i39j0j46j0l2j69i61l2.2373j1j4&sourceid=chrome&ie=UTF-8"
html = requests.get(convert_url, headers=headers)
soup = BeautifulSoup(html.content, 'html.parser')
curr_value = soup.find('div', class_= "b1hJbf")
curr_value = round(float(curr_value["data-exchange-rate"]),2)

def get_value_us(x):
    value = []
    for char in x:
        value.append(char)
    if value[-1] == "m":
        float_value = "".join(value[1:-1])
        return round(float(float_value)*curr_value,2)
    else:
        float_value = "".join(value[1:-3])
        return round((float(float_value)*curr_value)/1000,2)


In [6]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text.strip())

In [7]:
def fix_heights(df):
    heights = []
    
    for index, row in df.iterrows():
        if pd.notnull(row["Height"]):
            heights.append(row["Height"])
        else:
            nat = row["Nat"]
            position = row["position"]
            field_position = row["field_position"]
            comp_players = df.loc[((df["Nat"] == nat) & (df["position"] == position))]
            avg_df = round(comp_players["Height"].mean(skipna = True),2)
            row["Height"] = avg_df
            if pd.isnull(row["Height"]):
                comp_players2 = df.loc[((df["Nat"] == nat) & (df["field_position"] == field_position))]
                avg_df2 = round(comp_players2["Height"].mean(skipna = True),2)
                row["Height"] = avg_df2
                if pd.isnull(row["Height"]):
                    comp_players3 = df.loc[((df["position"] == position))]
                    avg_df3 = round(comp_players3["Height"].mean(skipna = True),2)
                    row["Height"] = avg_df3
            heights.append(row["Height"])
    
    
    df.height = heights

In [8]:
def scrape_league_data(leagues_df):
    team_id = 1
    team_ID = []
    links = []
    names = []
    logos = []
    squads = []
    foreigners = []
    total_MVs = []
    avg_MVs = []
    league_ID = []
    
    for index, row in leagues_df.iterrows():
        url = row["league_link"]
        league_name = row["league_name"]
        tier = row["tier"]
        country = row["country"]
        league_id = row["league_ID"]
        time.sleep(1.1)
        print(f"scraping: {country}_{tier}_{league_name}")
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.content, 'html.parser')
        htmltable = soup.find('table', class_= "items")


        results = htmltable.findAll("tr", class_ =["odd","even"])


        for result in results:
            features = result.findAll("td")
            links.append(("https://www.transfermarkt.com"+result.find("a", href=True)\
                          ["href"]+"/plus/1").replace("startseite", "kader"))
            logo = result.find("img", src=True)["src"]
            logo = logo.split("?")[0]
            logo = logo.replace("tiny", "header")
            logos.append(logo)
            name = features[1].text
            names.append(strip_accents(name))
            squad = features[3].text
            squads.append(squad)
            foreigner = features[5].text
            foreigners.append(foreigner)
            total_MV = get_value_us(features[6].text)
            total_MVs.append(total_MV)
#             print(features[7])
            avg_MV = get_value_us(features[7].text)
            avg_MVs.append(avg_MV)
            team_ID.append(team_id)
            league_ID.append(league_id)
            team_id = team_id + 1

            
#         break


    # Create a Dataframe and export to a .csv file
    df = pd.DataFrame(list(zip(team_ID, league_ID, names, squads, foreigners,avg_MVs, total_MVs, logos,links)), \
columns =["team_ID", "league_ID","club","squad", "foreigners", "avg_market_value_m", "total_MV_m",'Logo_img', "link_page"]) 
    df['league_ID'] = df['league_ID'].astype(int)
    df['team_ID'] = df['team_ID'].astype(int)

    df.to_csv(f'../data/teams_trmk.csv',index=False)




In [9]:
def scrape_team_data(teams_df):
    player_id = 1
    name = []
    player_page = []
    position = []
    f_posi = []
    Age = []
    Nat = []
    Height = []
    foot = []
    dt_joined = []
    prev_team = []
    contract_expires = []
    market_value = []
    team_ID = []
    players_ID = []
    

    df_league = pd.read_csv(teams_df)
    for index, row in df_league.iterrows():
        team_id = row["team_ID"]
        team = row["link_page"]
        team_name = team.split('/')[-8]
        print(team_name)
        time.sleep(randint(1,2))
        html = requests.get(team, headers=headers)
        soup = BeautifulSoup(html.content, 'html.parser')
        htmltable = soup.find('table', class_= "items")

        results = htmltable.findAll("tr", class_ =["odd","even"])


        for result in results:
            features = result.findAll("td")

            name_1 = features[2].find("img", alt=True)["alt"]

            player_page.append("https://www.transfermarkt.com" + features[3].find("a", href=True)["href"])

            position_1 = features[4].text
            
            if position_1 in ("Centre-Back", "Left-Back", "Right-Back", "Defender"):
                field_posit = "DEF"
            elif position_1 in ("Defensive Midfield", "Central Midfield" , "Right Midfield" , "Left Midfield" , \
            "Attacking Midfield" , "Midfield"):
                field_posit = "MID"
            elif position_1 in ("Left Winger" , "Right Winger" , "Centre-Forward" , "Second Striker" , "Forward"):
                field_posit = "ATT"
            else:
                field_posit = "GLK"

            try:
                age_1 = int((features[5].text.split("(",)[-1])[:-1])
            except:
                age_1 = ""

            nat = features[6].img["alt"]

            try:
                Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
            except:
                Height_1 = ""

            foot_1 = features[8].text

            dt_joined_1 = features[9].text
            try:
                dt_joined_1 = datetime.strptime(dt_joined_1, '%b %d, %Y').date()
            except:
                dt_joined_1 = ""

            try:
                prev_team_1 = features[10].img["alt"]
            except:
                prev_team_1 = "N.A."


            contract_expires_1 = features[11].text
            try:
                contract_expires_1 = datetime.strptime(contract_expires_1, '%d.%m.%Y').date()
            except:
                contract_expires_1 = ""

            try:
                market_value_1 = get_value_us(features[12].text[:-2])
            except:
                market_value_1 = 0

            name.append(strip_accents(name_1))
            position.append(position_1)
            f_posi.append(field_posit)
            Age.append(age_1)
            Nat.append(nat)
            Height.append(Height_1)
            foot.append(foot_1)
            dt_joined.append(dt_joined_1)
            prev_team.append(strip_accents(prev_team_1))
            contract_expires.append(contract_expires_1)
            market_value.append(market_value_1)
            team_ID.append(team_id)
            players_ID.append(player_id)
            player_id = player_id+1

#         time.sleep(randint(3,5))
        

    df = pd.DataFrame(list(zip(players_ID, team_ID, name, position,f_posi, Age,Nat, Height, foot,dt_joined,prev_team, contract_expires,\
                               market_value,player_page)), 
                      columns =["players_ID", "team_ID","name","position","field_position", "Age", "Nat","Height","foot",'dt_joined',"prev_team", \
                                "contract_expires", "market_value","player_page" ])
    df['players_ID'] = df['players_ID'].astype(int)
    df['team_ID'] = df['team_ID'].astype(int)
    df['dt_joined'] = pd.to_datetime(df['dt_joined'])
    df['contract_expires'] = pd.to_datetime(df['contract_expires'])
    
    fix_heights(df)

    df.to_csv(f"../data/players_trmk.csv" ,index=False)


In [10]:
scrape_league_data(leagues_df)

scraping: Brazil_1_Brazil Serie A
scraping: United States_1_MLS (US)
scraping: Mexico_1_Liga MX (MEX)
scraping: Argentina_1_Superliga (ARG)


In [11]:
teams_df = "../data/teams_trmk.csv"

scrape_team_data(teams_df)

flamengo-rio-de-janeiro
se-palmeiras-sao-paulo
gremio-foot-ball-porto-alegrense
corinthians-sao-paulo
fc-sao-paulo
atletico-mineiro
sc-internacional-porto-alegre
fc-santos
fluminense-football-club
vasco-da-gama-rio-de-janeiro
clube-atletico-paranaense
ec-bahia
botafogo-fr-rio-de-janeiro
clube-atletico-bragantino-sp-
goias-esporte-clube
coritiba-fc
ceara-sporting-club-ce-
sport-club-do-recife
fortaleza-esporte-clube
atletico-goianiense
atlanta-united-fc
los-angeles-football-club
inter-miami-cf
los-angeles-galaxy
toronto-fc
new-york-city-fc
seattle-sounders-fc
sporting-kansas-city
columbus-crew
portland-timbers
fc-cincinnati
fc-dallas
chicago-fire
d-c-united
montreal-impact
orlando-city-sc
new-england-revolution
new-york-red-bulls
houston-dynamo
colorado-rapids
vancouver-whitecaps
minnesota-united-fc
san-jose-earthquakes
philadelphia-union
real-salt-lake-city
nashville-mls-team
cf-monterrey
tigres-uanl
cf-america
cd-cruz-azul
deportivo-guadalajara
santos-laguna
club-tijuana
cf-pachuca
cl

C:\Users\joaog\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [12]:
# players_df = pd.read_csv("../data/players_trmk.csv")
# fix_heights(players_df)

## Save scraped data on SQL Database

In [13]:
from sqlalchemy import *
from sqlalchemy.schema import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import text
import contextlib 
import json

In [14]:
rds_connection_string = "postgres:PostgreSQL@localhost:5432/trfmk_scrape_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
metadata = MetaData()


In [15]:
leagues_df = pd.read_csv("../data/leagues_data.csv")
teams_df = pd.read_csv("../data/teams_trmk.csv")
players_df = pd.read_csv("../data/players_trmk.csv")

In [16]:
engine.execute(DropSchema('public', cascade = True))
engine.execute('CREATE SCHEMA IF NOT EXISTS public;')

players_df.to_sql(name = 'players',index_label= None, index= False, con=engine, if_exists='replace')
teams_df.to_sql(name = 'teams', index_label= None, index= False, con=engine, if_exists='replace')
leagues_df.to_sql(name = 'leagues', index_label= None, index= False, con=engine, if_exists='replace')

with engine.connect() as con:
    con.execute('ALTER TABLE leagues ADD CONSTRAINT pk_Leagues PRIMARY KEY ("league_ID")')
    con.execute('ALTER TABLE teams ADD CONSTRAINT pk_Teams PRIMARY KEY ("team_ID")')
    con.execute('ALTER TABLE players ADD CONSTRAINT pk_Players PRIMARY KEY ("players_ID")')
    con.execute('ALTER TABLE Teams ADD CONSTRAINT fk_Teams_league_ID FOREIGN KEY("league_ID") REFERENCES Leagues ("league_ID");')
    con.execute('ALTER TABLE Players ADD CONSTRAINT fk_Players_team_ID FOREIGN KEY("team_ID") REFERENCES Teams ("team_ID");')

In [17]:
# read database
pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)

,league_ID,country,tier,league_name,league_link,team_ID,club,squad,foreigners,avg_market_value_m,...,field_position,Age,Nat,Height,foot,dt_joined,prev_team,contract_expires,market_value,player_page
0,1,Brazil,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,32,2,4.44,...,GLK,35,Brazil,1.87,left,2017-07-16,Valencia CF,2020-12-31,3.30,https://www.transfermarkt.com/diego-alves/prof...
1,1,Brazil,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,32,2,4.44,...,GLK,21,Brazil,1.96,both,2019-08-31,Clube de Regatas do Flamengo U20,2023-09-30,0.53,https://www.transfermarkt.com/hugo-souza/profi...
2,1,Brazil,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,32,2,4.44,...,GLK,28,Brazil,1.94,right,2013-01-01,Clube de Regatas do Flamengo U20,2022-04-30,0.47,https://www.transfermarkt.com/cesar/profil/spi...
3,1,Brazil,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,32,2,4.44,...,GLK,24,Brazil,1.85,right,2016-01-01,Clube de Regatas do Flamengo U20,2020-12-31,0.12,https://www.transfermarkt.com/thiago-da-silva/...
4,1,Brazil,1,Brazil Serie A,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,32,2,4.44,...,GLK,22,Brazil,1.88,right,2017-01-01,Clube de Regatas do Flamengo U20,2022-12-31,0.06,https://www.transfermarkt.com/gabriel-batista/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,4,Argentina,1,Superliga (ARG),https://www.transfermarkt.com/primera-division...,88,Arsenal Futbol Club,12,1,0.31,...,MID,31,Argentina,1.86,right,2018-07-25,CD Moron,2021-12-31,0.24,https://www.transfermarkt.com/emiliano-mendez/...
2480,4,Argentina,1,Superliga (ARG),https://www.transfermarkt.com/primera-division...,88,Arsenal Futbol Club,12,1,0.31,...,MID,21,Argentina,NaN,-,2019-06-25,Arsenal Futbol Club II,2022-06-30,0.12,https://www.transfermarkt.com/leonel-picco/pro...
2481,4,Argentina,1,Superliga (ARG),https://www.transfermarkt.com/primera-division...,88,Arsenal Futbol Club,12,1,0.31,...,MID,20,Argentina,1.71,left,2018-07-01,Arsenal Futbol Club U20,2021-06-30,0.91,https://www.transfermarkt.com/lucas-necul/prof...
2482,4,Argentina,1,Superliga (ARG),https://www.transfermarkt.com/primera-division...,88,Arsenal Futbol Club,12,1,0.31,...,ATT,24,Argentina,1.95,right,2018-07-01,Arsenal Futbol Club II,2021-06-30,0.47,https://www.transfermarkt.com/facundo-pons/pro...


### Save query as JSON file

In [18]:
pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)\
.to_json(r'../dashboard/static/data/data.json', orient='records')